# 
1)  THe CHAT BOT is designed to questions regarding the CSVs provided
2)  OPEN AI reterival chains has been used
3)  THe Requirements.txt file has been provided
4)  The APi key will be removed for privacy reasons kindly replace it with any to see the results live
5)  Change the file path for the data sets

# IMPORT SECTION

In [16]:
from langchain.document_loaders import CSVLoader
from langchain.indexes import VectorstoreIndexCreator
from langchain.chains import RetrievalQA
from langchain.llms import OpenAI  
import warnings
warnings.filterwarnings("ignore")
import os
from langchain.agents.agent_types import AgentType
from langchain_experimental.agents.agent_toolkits import create_csv_agent
import pandas as pd
import sys

# API KEY

In [4]:
import os

#using OPENAI_API key here for usage
#we will use retreival chain here as both the data sets have been cleaned and preprocessed for better index management
# Retreival chain is better of with data set of such complexity

os.environ["OPENAI_API_KEY"] = ""

# CREATING THE FILTER WORDS

In [23]:
df1 = pd.read_csv("E:\Propertyloop\data\holdings.csv")
df2 = pd.read_csv("E:\Propertyloop\data\Trades.csv")
holdings_filter_words = set(df1.columns)
trades_filter_words = set(df2.columns)
combined_filter_words = holdings_filter_words.union(trades_filter_words)
filter_word_list = list(combined_filter_words)

In [32]:
df1.head()

,AsOfDate,OpenDate,CloseDate,ShortName,PortfolioName,StrategyRefShortName,Strategy1RefShortName,Strategy2RefShortName,CustodianName,DirectionName,SecurityId,SecurityTypeName,SecName,StartQty,Qty,StartPrice,Price,StartFXRate,FXRate,MV_Local,MV_Base,PL_DTD,PL_QTD,PL_MTD,PL_YTD
0,2023-01-08,04/03/20,Undefined,Garfield,Garfield,Default,Asset,DefaultS2,Well Prime,Long,273098,Bond,EJ0445951,592000.0,592000.0,96.0,96.0,1.33,1.33,568320.00,7.558656e+05,92.5040,10833.7294,92.5040,41054.5854
1,2023-01-08,04/03/20,Undefined,Garfield,Garfield,Default,Asset,DefaultS2,Well Prime,Long,273098,Bond,EJ0445951,88.0,88.0,96.0,96.0,1.33,1.33,84.48,1.123584e+02,0.0138,1.6104,0.0138,6.1027
2,2023-01-08,04/03/20,Undefined,Garfield,Garfield,Default,Asset,DefaultS2,Well Prime,Long,273098,Bond,EJ0445951,787500.0,787500.0,96.0,96.0,1.33,1.33,756000.00,1.005480e+06,123.0523,14411.4221,123.0523,54612.3074
3,2023-01-08,04/03/20,Undefined,Garfield,Garfield,Default,Asset,DefaultS2,Well Prime,Long,273098,Bond,EJ0445951,505000.0,505000.0,96.0,96.0,1.33,1.33,484800.00,6.447840e+05,78.9097,9241.6104,78.9097,35021.2257
4,2023-01-08,04/03/20,Undefined,Heather,Heather,Default,Asset,DefaultS2,Well Prime,Long,273098,Bond,EJ0445951,508000.0,508000.0,96.0,96.0,1.33,1.33,487680.00,6.486144e+05,79.3785,9296.5110,79.3785,35229.2726


In [38]:
df2.head()

,id,RevisionId,AllocationId,TradeTypeName,SecurityId,SecurityType,Name,Ticker,CUSIP,ISIN,TradeDate,SettleDate,Quantity,Price,TradeFXRate,Principal,Interest,TotalCash,AllocationQTY,AllocationPrincipal,AllocationInterest,AllocationFees,AllocationCash,PortfolioName,CustodianName,StrategyName,Strategy1Name,Strategy2Name,Counterparty,AllocationRule,IsCustomAllocation
0,3489863,2,3460886,Buy,270471,Equity,Berry Brand 4/11 Equity,IBM,459200101,US4592001014,00:00.0,00:00.0,500000,14.0,NaN,7.000000e+06,0.0,7.002800e+06,500000.0,7.000000e+06,0.0,2800.00,7.002800e+06,HoldCo 1,JP MORGAN SECURITIES LLC,Default,DefaultS1,DefaultS2,ABGS,Single Fund Rule - HoldCo 1,1
1,3489864,1,3460887,Sell,270471,Equity,Berry Brand 4/11 Equity,IBM,459200101,US4592001014,00:00.0,00:00.0,500000,14.0,NaN,7.000000e+06,0.0,6.999871e+06,500000.0,7.000000e+06,0.0,128.80,6.999871e+06,HoldCo 1,JP MORGAN SECURITIES LLC,Default,DefaultS1,DefaultS2,ABGS,Single Fund Rule - HoldCo 1,0
2,3496826,1,3462756,Sell,290063,Equity,META-US,META,30303M102,US30303M1027,00:00.0,00:00.0,23644323,108.0,NaN,2.553587e+09,0.0,2.553540e+09,23644323.0,2.553587e+09,0.0,46985.99,2.553540e+09,HoldCo 3,CITIGROUP GLOBAL MARKETS INC.,Default,DefaultS1,DefaultS2,ABGS,Single Fund Rule - HoldCo 3,0
3,3496828,3,3462769,Buy,290067,Equity,SPOT-US,SPOT,459200101,LU1778762911,00:00.0,00:00.0,1098229,1.0,NaN,1.098229e+06,0.0,1.098249e+06,1098229.0,1.098229e+06,0.0,20.20,1.098249e+06,HoldCo 11,Goldman Sachs International,Default,Asset,DefaultS2,ABGS,Single Fund Rule - HoldCo 11,1
4,3496829,4,3462770,Buy,290067,Equity,SPOT-US,SPOT,459200101,LU1778762911,00:00.0,00:00.0,1647344,2.0,NaN,3.294688e+06,0.0,3.294749e+06,1647344.0,3.294688e+06,0.0,60.62,3.294749e+06,HoldCo 11,Goldman Sachs International,Default,Asset,DefaultS2,ABGS,Single Fund Rule - HoldCo 11,1


# LOADING THE CSV

In [7]:
# Define file paths for  data 
holdings_file_path = "E:\Propertyloop\data\holdings.csv"
trades_file_path = "E:\Propertyloop\data\Trades.csv"

# Load data sets using CSV loaders
holdings_loader = CSVLoader(file_path=holdings_file_path)
trades_loader = CSVLoader(file_path=trades_file_path)

# CREATING INDEX FOR SEMANTIC SEARCH

In [22]:
# Creating an index using the loaded documents (merged)
index_creator = VectorstoreIndexCreator()
docsearch = index_creator.from_loaders([holdings_loader, trades_loader])

# THE CHATBOT

In [39]:


# Create a RetrievalQA chain (merged retriever)
chain = RetrievalQA.from_chain_type(
    llm=OpenAI(),chain_type="stuff",retriever=docsearch.vectorstore.as_retriever(),input_key="question"
)

while True:
  # User input for the query
  print("Type exit to close the bot")
  query = input(f"Ask Your chatbot: ")
  print(query)
  if query == 'exit':
    print('Exiting')
    break
  
  # filter checking the query

#Preprocess the query and check if it contains filter words from both datasets
 



  response = chain({"question": query})

    # Handle the response from the LLM
  if response['result']:
        response_contains_filter_word = any(word in response['result'].lower() for word in filter_word_list)
        if response_contains_filter_word:
          print("Response:", response['result'])
        else:
            print("Response:", response['result'])
            print("is this right?")
        
  else:
      print("I dont Know")
     
        


Type exit to close the bot
Provide the total "Principal" across all entries.?
Response:  The total "Principal" across all entries is $6,971,800.
is this right?
Type exit to close the bot
How many entries have a "PL_MTD" greater than 10000?
Response:  I don't know.
is this right?
Type exit to close the bot
List all unique "CustodianName" in the DataFrame.?
Response:  JP MORGAN SECURITIES LLC, JPHQ, Well Prime, Default Custodian
is this right?
Type exit to close the bot
Can you provide the "SettleDate" for the entry with "CUSIP" '459200101'?
Response:  I don't know.
is this right?
Type exit to close the bot
How many entries have a "TradeTypeName" of 'Buy'?
Response:  There are four entries with a "TradeTypeName" of 'Buy'.
is this right?
Type exit to close the bot
Can you provide the "ISIN" for the entry with "Ticker" 'SPOT'?
Response:  LU1778762911
is this right?
Type exit to close the bot
Tell me about buggati?
Response:  I'm sorry, I don't have any information about buggati. The contex